In [3]:
USE {
    dependencies {
        implementation("com.fasterxml.jackson.module:jackson-module-kotlin:2.15.3")
    }
}

In [23]:
import com.fasterxml.jackson.databind.ObjectMapper
import java.io.BufferedReader
import java.io.InputStreamReader
import java.net.HttpURLConnection
import java.net.URL

val domain = "http://localhost:8080"
fun readUrl(urlString: String): String {
    var reader: BufferedReader? = null
    var result = "{}"
    try {
        val url = URL(urlString)
        reader = BufferedReader(InputStreamReader(url.openStream()))
        val buffer = StringBuffer()
        var read: Int
        val chars = CharArray(1024)
        while (reader.read(chars).also { read = it } != -1) buffer.append(chars, 0, read)
        result = buffer.toString()
    } finally {
        reader?.close()
    }
    return result
}
val problemsCount = readUrl("${domain}/problems").count { it == '}' }
problemsCount

90

In [24]:
val ids = (1..getProblemsCount()).toList().sortedByDescending { abs(it - 18) }
ids

[90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 1, 35, 2, 34, 3, 33, 4, 32, 5, 31, 6, 30, 7, 29, 8, 28, 9, 27, 10, 26, 11, 25, 12, 24, 13, 23, 14, 22, 15, 21, 16, 20, 17, 19, 18]

In [14]:
import com.fasterxml.jackson.annotation.JsonProperty
import java.util.stream.Collectors
data class Task(
    @JsonProperty("room_width")
    val room_width: Double,
    @JsonProperty("room_height")
    val room_height: Double,
    @JsonProperty("stage_width")
    val stage_width: Double,
    @JsonProperty("stage_height")
    val stage_height: Double,
    @JsonProperty("stage_bottom_left")
    val stage_bottom_left: List<Double>,
    @JsonProperty("musicians")
    val musicians: List<Int>,
    @JsonProperty("attendees")
    val attendees: List<Attendee>,
    @JsonProperty("pillars")
    val pillars: List<Pillars>
) {
    companion object {
        fun parse(json: String): Task {
            return ObjectMapper().readValue(json, Task::class.java)
        }
    }
}

data class Attendee(
    @JsonProperty("x")
    val x: Double,
    @JsonProperty("y")
    val y: Double,
    @JsonProperty("tastes")
    val tastes: List<Double>
)

data class Pillars(
    @JsonProperty("center")
    val center: List<Double>,
    @JsonProperty("radius")
    val radius: Double
)

data class Solve(
    @JsonProperty("placements")
    val placements: List<Point>,
    @JsonProperty("volumes")
    val volumes: List<Double>? = (1..placements.size).map { 10.0 }
) {
    companion object {
        fun parse(json: String): Solve {
            return ObjectMapper().readValue(json, Solve::class.java)
        }
    }
}

data class Point(
    @JsonProperty("x")
    val x: Double,
    @JsonProperty("y")
    val y: Double
) {
    operator fun plus(p: Point) = Point(x + p.x, y + p.y)

    operator fun minus(p: Point) = Point(x - p.x, y - p.y)

    operator fun times(d: Double) = Point(x * d, y * d)

    operator fun div(d: Double) = Point(x / d, y / d)

    infix fun scalar(p: Point) = p.x * x + p.y * y

    infix fun dist(p: Point) = sqrt((this - p).sqrSize())

    fun sqrSize() = x * x + y * y

    fun norm() = this / sqrt(sqrSize())
}


fun getCells(problem: Task, scale: Boolean = true): List<Point> {
    val cells = (0..(problem.stage_width.toInt() - 20) step 10).flatMap { x ->
        (0..(problem.stage_height.toInt() - 20) step 10).map { y ->
            Point(
                x.toDouble() + problem.stage_bottom_left[0] + 10.0,
                y.toDouble() + problem.stage_bottom_left[1] + 10.0
            )
        }
    }
    if (!scale) {
        return cells
    }
    val coef = getCoef(problem, cells)
    return cells.map {
        val x = if (coef.x.isNaN()) {
            it.x
        } else {
            (((it.x - (problem.stage_bottom_left[0] + 10.0)) * coef.x + (problem.stage_bottom_left[0] + 10.0)) * 1000).roundToLong() / 1000.0
        }
        val y = if (coef.y.isNaN()) {
            it.y
        } else {
            (((it.y - (problem.stage_bottom_left[1] + 10.0)) * coef.y + (problem.stage_bottom_left[1] + 10.0)) * 1000).roundToLong() / 1000.0
        }
        Point(x, y)
    }
}

fun getCoef(problem: Task, cells: List<Point>): Point {
    val coefX =
        (problem.stage_width - 20) / (cells.maxOfOrNull { it.x }!! - cells.minOfOrNull { it.x }!!)
    val coefY =
        (problem.stage_height - 20) / (cells.maxOfOrNull { it.y }!! - cells.minOfOrNull { it.y }!!)
    val x = if (coefX.isNaN()) {
        1.0
    } else {
        coefX
    }
    val y = if (coefY.isNaN()) {
        1.0
    } else {
        coefY
    }
    return Point(x, y)
}

fun List<Point>.filterBorder(problem: Task, dist: Int = 1): List<Point> {
    return filter {
        val a = min(
            abs(it.x - problem.stage_bottom_left[0]),
            abs(problem.stage_bottom_left[0] + problem.stage_width - it.x)
        )
        val b = min(
            abs(it.y - problem.stage_bottom_left[1]),
            abs(problem.stage_bottom_left[1] + problem.stage_height - it.y)
        )
        min(a, b) < (dist + 1) * 10.0
    }
}

fun <T, R> Collection<T>.pmap(mapper: (T) -> R): List<R> {
    return this.stream().parallel().map { mapper(it) }.collect(Collectors.toList())
}


In [15]:
fun getPlaceFor(problem: Task): Solve {
    val sourceCells = getCells(problem).shuffled()
    var dist = 1
    val borderCells = sourceCells.filterBorder(problem, dist = dist)
    val cells = borderCells.toMutableSet()
    println("cellssize ${cells.size}")
//        println("cells ${cells}")
    val mPoints = mutableSetOf<Point>()

    val musicians = mutableMapOf<Int, MutableSet<Int>>()
    problem.musicians.forEachIndexed { i, m ->
        val set = musicians[m] ?: mutableSetOf()
        set += i
        musicians[m] = set
    }

    val pillars = problem.pillars.map { Pillar(it.center[0], it.center[1], it.radius) }.toMutableList()
    val result = mutableMapOf<Int, Point>()

    val data = borderCells.pmap { mPos ->
        val otherPillars = borderCells.filter { it != mPos }.map { Pillar(it.x, it.y, 5.0) }
        musicians.keys.map { instr ->
            val score = scoring.calcWithPillars(problem, instr, mPos, pillars + otherPillars)
            score to instr
        }.map { Triple(mPos, it.first, it.second) }
    }.flatten()

    while (mPoints.size < problem.musicians.size && cells.isNotEmpty()) {
        val (cell, score, instr) = data.filter { it.first !in mPoints && it.third in musicians }
            .maxByOrNull { (_, l, _) -> l } ?: break
        if (score <= 0) {
            break
        }
        cells -= cell
        musicians[instr]!!.let { set ->
            val index = set.first()
            println("found best place for $instr on $cell at $score")
            mPoints += cell
            result[index] = cell
            set.remove(index)
            if (set.isEmpty()) {
                musicians.remove(instr)
            }
        }
    }

    musicians.values.flatten().let { notFilled ->
        if (notFilled.isNotEmpty()) {
            println("not filled ${notFilled.size} cells")
            val scores = scoring.calcInfluencesWithoutVolume(problem, (0 until problem.musicians.size).map { result[it] ?: Point(-1000000000.0, -1000000000.0) })
                .mapIndexed { i, it -> i to it }.sortedByDescending { it.second }
            val map = result.entries.associate { it.value to scores[it.key].second }
            val notUsed =
                LinkedList(sourceCells.filter { it !in mPoints }.sortedByDescending { p -> mPoints.sumOf { map[it]!!.toDouble() / (p dist it) } })
            notFilled.forEach {
                result[it] = notUsed.removeFirst()
            }
        }
    }


    return addVolume(problem, result.entries.sortedBy { it.key }.map { it.value })
}

fun addVolume(problem: Task, placement: List<Point>): Solve {
    val influenceCoeff = scoring.calcInfluencesWithoutVolume(problem, placement)
        .map { if (it > 0) 10.0 else 0.0 }
    return Solve(placement, influenceCoeff)
}

Line_16.jupyter.kts (22:41 - 47) Unresolved reference: Pillar
Line_16.jupyter.kts (26:68 - 74) Unresolved reference: Pillar
Line_16.jupyter.kts (28:25 - 32) Unresolved reference: scoring
Line_16.jupyter.kts (30:30 - 32) Unresolved reference: it
Line_16.jupyter.kts (30:40 - 42) Unresolved reference: it
Line_16.jupyter.kts (34:14 - 18) Unreachable code
Line_16.jupyter.kts (34:20 - 25) Unreachable code
Line_16.jupyter.kts (34:27 - 32) Unreachable code
Line_16.jupyter.kts (34:36 - 35:53) Function 'component1()' is ambiguous for this expression: 
public inline operator fun <T> Array<out TypeVariable(T)>.component1(): TypeVariable(T) defined in kotlin.collections
public inline operator fun BooleanArray.component1(): Boolean defined in kotlin.collections
public inline operator fun ByteArray.component1(): Byte defined in kotlin.collections
public inline operator fun CharArray.component1(): Char defined in kotlin.collections
public inline operator fun DoubleArray.component1(): Double defined in

In [ ]:


fun test2(id: Int) {
    val problem = Task.parse(readUrl("$domain/problem/$id"))
    val solution = getPlaceFor(problem)
    val score = scoring.calc(problem, solution)
    URL("$domain/add/$id?calc=true").send(solution)
    println("score for $id: $score")
}